In [26]:
import numpy as np
import cv2
import pandas as pd 
from imutils import paths
import os
from PIL import Image
from sklearn.preprocessing import LabelBinarizer

In [15]:
from sklearn.model_selection import train_test_split

In [4]:
tom_dir=r'C:\Users\Atharva\Desktop\SIH\archive\PlantVillage\Tomato'

In [5]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
    return image_paths,labels

In [13]:
df = pd.DataFrame()
df['image'], df['label'] = createdataframe(tom_dir)

In [14]:
df.head(5)

,image,label
0,C:\Users\Atharva\Desktop\SIH\archive\PlantVill...,Tomato_Bacterial_spot
1,C:\Users\Atharva\Desktop\SIH\archive\PlantVill...,Tomato_Bacterial_spot
2,C:\Users\Atharva\Desktop\SIH\archive\PlantVill...,Tomato_Bacterial_spot
3,C:\Users\Atharva\Desktop\SIH\archive\PlantVill...,Tomato_Bacterial_spot
4,C:\Users\Atharva\Desktop\SIH\archive\PlantVill...,Tomato_Bacterial_spot


In [23]:
df.shape

(16012, 3)

In [24]:
df.columns

Index(['image', 'label', 'path_exists'], dtype='object')

In [20]:
def check_image_paths(df, column_name='image'):
    # Check if the path exists for each row in the 'image' column
    df['path_exists'] = df[column_name].apply(lambda x: os.path.exists(x))

    # Get rows where the path doesn't exist
    missing_images = df[~df['path_exists']]

    if not missing_images.empty:
        print("Some image paths are incorrect or the images are missing.")
        print(missing_images[[column_name]])
    else:
        print("All image paths are correct and accessible.")

# Example usage
check_image_paths(df, column_name='image')

All image paths are correct and accessible.


In [21]:
def preprocess_image(image_path, img_size):
    # Load the image from the file, resize it to the target size, and normalize pixel values
    image_list=[]
    for images in image_path:
        image = cv2.imread(images)
        if image is None:
            continue
            
        image = cv2.resize(image, img_size)
        image = image.astype('float32') / 255.0
        image_list.append(image)
    image_list=np.array(image_list)
    print(image_list.shape)
    return image_list

In [25]:
img_size = (150, 150)
X = preprocess_image(df['image'],img_size)

(16011, 150, 150, 3)


In [27]:
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(df['label'])

In [28]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [16011, 16012]

In [8]:
# image=cv2.imread(train_df['image'][0])
# window_name="image"
# cv2.imshow(window_name,image)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [12]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_df.class_indices), activation='softmax')  # Number of classes
])


c:\Users\Atharva\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: 'DataFrame' object has no attribute 'class_indices'